In [1]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.0/608.0 kB 5.4 MB/s eta 0:00:00


In [2]:
import torch
import cv2
import os
from ultralytics import YOLO

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [14]:
def load_yolov5_model(weights_path):
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=weights_path)
    return model

In [15]:
def draw_bounding_boxes(label_file, results):
    image = cv2.imread("/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg")
    boxes = results.xyxy[0].detach().cpu().numpy()

    with open(label_file, 'w') as f:
      for i,box in enumerate(boxes):
        im=image.copy()
        x1, y1, x2, y2 = box[:4].astype(int)
        coord=(x1,y1,x2,y2)
        im=im[y1:y2,x1:x2]
        # os.chdir(image_path)
        # cv2.imwrite(f"img{i}.jpg",im)
        # os.chdir("/content/drive")
        print(coord)
        label = f"{results.names[int(box[5])]}"
        f.write(f"{x1} {y1} {x2} {y2}\n")

In [16]:
def load_model(model_path):
    model=YOLO(model=model_path)
    return model

In [17]:
def extract_roi(image_path,coord_list,i):
    image=cv2.imread(image_path)
    im=image.copy()

    x1, y1, x2, y2 = coord_list
    roi = im[y1:y2, x1:x2]

    cv2.imwrite(f"roi{i}.jpg",roi)
    return roi

In [18]:
def extract_labels(roi,model,i):



    results=model.predict(source=f"roi{i}.jpg",conf=0.5,save=False,save_txt=True)
    contours=[]
    for result in results:
        boxes = result.boxes

    i = 0
    label_string_list =[]
    num_objects=0
    num_emptys=0
    while i < len(boxes.xyxy.tolist()):

      bbox = boxes.xyxy.tolist()[i]
      class_id = int(boxes.cls.tolist()[i])
      if class_id==0:
        num_objects +=1
      else:
        num_emptys +=1
      i+=1
    return num_objects,num_emptys
    print("Objects:",num_objects)
    print("emptys:",num_emptys)

In [19]:
weights_path = "/content/drive/MyDrive/phase_2.pt"
image=cv2.imread("/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg")
model = load_yolov5_model(weights_path)
results=model(image)
label_file="/content/drive/MyDrive/data/output"
draw_bounding_boxes(label_file, results)
model_path="/content/drive/MyDrive/best_detect.pt"
model=load_model(model_path)
results_dict={}
output_path=label_file
bounding_box_list=[]
with open(output_path, 'r') as f:
   for line in f:
    bounding_box = [int(coord) for coord in line.strip().split()]
    bounding_box_list.append(bounding_box)
destination_path = "/content/drive/MyDrive/Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg"
results_dict["image_name"] = destination_path.split("/")[-1]
results_dict["coordinates"]=bounding_box_list
counts_list=[]


for i in range(len(bounding_box_list)):
  roi = extract_roi(destination_path,bounding_box_list[i],i)
  num_objs,num_empts=extract_labels(roi,model,i)
  counts_list.append([num_objs,num_empts])
results_dict["count"] = counts_list
print(results_dict)


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-26 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients
Adding AutoShape... 


0
(2859, 334, 4396, 1304)
1
(782, 1769, 1607, 2196)
2
(2875, 1448, 4471, 2443)
3
(2881, 2654, 4582, 3456)
4
(694, 2623, 1553, 3123)
5
(791, 705, 1634, 1487)
6
(1651, 1731, 2786, 2234)
7
(1, 2382, 613, 3039)
8
(1600, 2673, 2798, 3332)
9
(3, 1650, 718, 2250)
10
(70, 860, 771, 1534)



image 1/1 /content/roi0.jpg: 416x640 4 objects, 13.7ms
Speed: 3.1ms preprocess, 13.7ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict3
1 label saved to runs/detect/predict3/labels

image 1/1 /content/roi1.jpg: 352x640 3 objects, 11.5ms
Speed: 1.9ms preprocess, 11.5ms inference, 1.5ms postprocess per image at shape (1, 3, 352, 640)
Results saved to runs/detect/predict3
2 labels saved to runs/detect/predict3/labels

image 1/1 /content/roi2.jpg: 416x640 3 objects, 13.6ms
Speed: 2.2ms preprocess, 13.6ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict3
3 labels saved to runs/detect/predict3/labels

image 1/1 /content/roi3.jpg: 320x640 3 objects, 11.4ms
Speed: 1.8ms preprocess, 11.4ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)
Results saved to runs/detect/predict3
4 labels saved to runs/detect/predict3/labels

image 1/1 /content/roi4.jpg: 384x640 3 objects, 11.8

{'image_name': 'Trends-Seawoods-NaviMumbai_train_17-09-2022_0003.jpg', 'coordinates': [[2859, 334, 4396, 1304], [782, 1769, 1607, 2196], [2875, 1448, 4471, 2443], [2881, 2654, 4582, 3456], [694, 2623, 1553, 3123], [791, 705, 1634, 1487], [1651, 1731, 2786, 2234], [1, 2382, 613, 3039], [1600, 2673, 2798, 3332], [3, 1650, 718, 2250], [70, 860, 771, 1534]], 'count': [[4, 0], [3, 0], [3, 0], [3, 0], [3, 0], [5, 0], [3, 0], [6, 0], [3, 0], [10, 0], [14, 0]]}
